In [1]:
import torch
from tqdm.auto import tqdm
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM,AutoTokenizer

config = PeftConfig.from_pretrained("Rajan/training_run")
tokenizer = AutoTokenizer.from_pretrained("NumbersStation/nsql-350M")
base_model = AutoModelForCausalLM.from_pretrained("NumbersStation/nsql-350M")
model = PeftModel.from_pretrained(base_model, "Rajan/training_run")
model = model.cuda()

In [2]:
import re
import random
import pandas as pd
from datasets import load_dataset

# load the dataset
dataset = load_dataset("NumbersStation/NSText2SQL",split="train")


# convert the dataset into a pandas dataframe
df = pd.DataFrame(dataset)

def count_tables(instruction: str)-> int:
    """ Count the number of CREATE TABLE statements in the instruction """
    return len(re.findall(r'CREATE TABLE', instruction))


def replace_database(instruction:str)->str:
    """ Replace the valid SQLite database with a random database """
    databases = ['SQLite', 'MySQL', 'PostgreSQL']
    chosen_db = random.choice(databases)
    return re.sub(r'Using valid SQLite', f'Using valid {chosen_db}', instruction)

# filter the dataframe so that we have at least 3 tables and at most 10 tables.
filtered_df = df[(df['instruction'].apply(count_tables) > 3) & (df['instruction'].apply(count_tables) < 10)]

# add MySQL and PostgreSQL to the prompt.
filtered_df['instruction'] = filtered_df['instruction'].apply(replace_database)


# Calculate the number of rows removed
rows_removed = df.shape[0] - filtered_df.shape[0]

# Calculate the percentage of rows removed
percentage_removed = (rows_removed / df.shape[0]) * 100

# Print the results
print(f"Number of rows removed: {rows_removed}")
print(f"Percentage of rows removed: {percentage_removed:.2f}%")

del dataset

Using the latest cached version of the dataset since NumbersStation/NSText2SQL couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/zeus/.cache/huggingface/datasets/NumbersStation___ns_text2_sql/default/0.0.0/e77eeb9e172a1734493ee5bc63b883e9e831c6cf (last modified on Sat Jul 27 16:06:39 2024).


Number of rows removed: 232083
Percentage of rows removed: 80.23%


/tmp/ipykernel_12988/2163250706.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['instruction'] = filtered_df['instruction'].apply(replace_database)


In [3]:
from datasets import Dataset

filtered_dataset = Dataset.from_pandas(filtered_df)

filtered_dataset = filtered_dataset.remove_columns("__index_level_0__")


# do the train test split 
filtered_dataset = filtered_dataset.train_test_split(test_size=0.3)


In [4]:
test_datset = filtered_dataset["test"]
dataset = test_datset.select(range(50))

In [5]:
from tqdm.auto import  tqdm

def extract_and_correct_sql(text, correct=False):
    lines = text.splitlines()

    start_index = 0
    for i, line in enumerate(lines):
        if line.strip().upper().startswith("SELECT"):
            start_index = i
            break

    generated_sql = "\n".join(lines[start_index:])

    if correct:
        if not generated_sql.strip().endswith(";"):
            generated_sql = generated_sql.strip() + ";"

    return generated_sql

# Function to generate SQL and extract SELECT command
def generate_and_extract(instruction):
    text = f"{instruction}\nSELECT"
    model_input = tokenizer(text, return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_input, max_new_tokens=100)

    torch.cuda.empty_cache()
    
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    select_commands = extract_and_correct_sql(generated_text,correct=True)
    
    return select_commands if select_commands else ""

# Generate predictions and create a DataFrame
data = []
for row in tqdm(dataset):
    instruction = row["instruction"]
    output = row["output"]
    predicted = generate_and_extract(instruction)
    data.append({"instruction": instruction, "output": output, "predicted": predicted})

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("final_dataframe.csv", index=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [6]:
import pandas as pd
from sklearn.metrics import precision_score, accuracy_score

df.reset_index(inplace=True)


def tokenize(text):
    return text.split()

# Initialize counts
total_tp = 0
total_fp = 0
total_fn = 0

for i in range(len(df)):
    output_tokens = set(tokenize(df.loc[i, 'output']))
    predicted_tokens = set(tokenize(df.loc[i, 'predicted']))
    
    tp = len(output_tokens & predicted_tokens)
    fp = len(predicted_tokens - output_tokens)
    fn = len(output_tokens - predicted_tokens)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

# Calculate Precision and Accuracy
precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
accuracy = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0


print(f'Precision: {precision}')
print(f'Accuracy: {accuracy}')

Precision: 0.9174434087882823
Accuracy: 0.7162162162162162


In [7]:
# Initialize counts
total_tp = 0
total_fp = 0
total_fn = 0
total_tokens = 0

for i in range(len(df)):
    output_tokens = set(tokenize(df.loc[i, 'output']))
    predicted_tokens = set(tokenize(df.loc[i, 'predicted']))
    
    tp = len(output_tokens & predicted_tokens)
    fp = len(predicted_tokens - output_tokens)
    fn = len(output_tokens - predicted_tokens)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn
    total_tokens += len(output_tokens)

# Calculate Precision, Recall, F1 Score, and Accuracy
precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = total_tp / total_tokens if total_tokens > 0 else 0

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'Accuracy: {accuracy}')

Precision: 0.9174434087882823
Recall: 0.7162162162162162
F1 Score: 0.804436660828955
Accuracy: 0.7162162162162162
